In [4]:
INSTANCE_NAME="instance-1"
REGION='us-central1'
ZONE='us-central1-c'
PROJECT_NAME="information-retrieval-assg3"
IP_NAME= PROJECT_NAME + "-ip"
GOOGLE_ACCOUNT_NAME="eithanm" # without the @post.bgu.ac.il or @gmail.com part

# 0. Install Cloud SDK on your local machine or using Could Shell
# check that you have a proper active account listed
!gcloud auth list 
# check that the right project and zone are active
!gcloud config list




                  Credentialed Accounts
ACTIVE  ACCOUNT
*       556037941133-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`

[core]
account = 556037941133-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = information-retrieval-assg3

Your active configuration is: [default]


In [ ]:
# if not set them
# gcloud config set project $PROJECT_NAME
!gcloud config set compute/zone $ZONE

In [6]:
# 1. Set up public IP
!gcloud compute addresses create $IP_NAME --project=$PROJECT_NAME --region=$REGION
!gcloud compute addresses list

Created [https://www.googleapis.com/compute/v1/projects/information-retrieval-assg3/regions/us-central1/addresses/information-retrieval-assg3-ip].
NAME                            ADDRESS/RANGE   TYPE      PURPOSE  NETWORK  REGION       SUBNET  STATUS
information-retrieval-assg3-ip  34.123.208.148  EXTERNAL                    us-central1          RESERVED


In [9]:
# note the IP address printed above, that's your extrenal IP address.
# Enter it here: 
INSTANCE_IP="34.123.208.148"

In [7]:
# 2. Create Firewall rule to allow traffic to port 8080 on the instance
!gcloud compute firewall-rules create default-allow-http-8080 \
  --allow tcp:8080 \
  --source-ranges 0.0.0.0/0 \
  --target-tags http-server

Creating firewall...⠹Created [https://www.googleapis.com/compute/v1/projects/information-retrieval-assg3/global/firewalls/default-allow-http-8080].
Creating firewall...done.                                                      
NAME                     NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
default-allow-http-8080  default  INGRESS    1000      tcp:8080        False


In [12]:
%cd /home/dataproc
!ls

/home/dataproc
Untitled.ipynb	       run_frontend_in_gcp.sh  startup_script_gcp.sh
inverted_index_gcp.py  search_backend.py
page_views_dict.pkl    search_frontend.py


In [13]:
# 3. Create the instance. Change to a larger instance (larger than e2-micro) as needed.
!gcloud compute instances create $INSTANCE_NAME \
  --zone=$ZONE \
  --machine-type=e2-micro \
  --network-interface=address=$INSTANCE_IP,network-tier=PREMIUM,subnet=default \
  --metadata-from-file startup-script=startup_script_gcp.sh \
  --scopes=https://www.googleapis.com/auth/cloud-platform \
  --tags=http-server

Created [https://www.googleapis.com/compute/v1/projects/information-retrieval-assg3/zones/us-central1-c/instances/instance-1].
NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
instance-1  us-central1-c  e2-micro                   10.128.0.56  34.123.208.148  RUNNING


In [15]:
# monitor instance creation log using this command. When done (4-5 minutes) terminate using Ctrl+C
!gcloud compute instances tail-serial-port-output $INSTANCE_NAME --zone $ZONE

[=3h[=3hCSM BBS Table full.
BdsDxe: loading Boot0001 "UEFI Google PersistentDisk " from PciRoot(0x0)/Pci(0x3,0x0)/Scsi(0x1,0x0)
BdsDxe: starting Boot0001 "UEFI Google PersistentDisk " from PciRoot(0x0)/Pci(0x3,0x0)/Scsi(0x1,0x0)

UEFI: Attempting to start image.
Description: UEFI Google PersistentDisk 
FilePath: PciRoot(0x0)/Pci(0x3,0x0)/Scsi(0x1,0x0)
OptionNumber: 1.

Welcome to GRUB!

    Booting `Debian GNU/Linux'Booting `Debian GNU/Linux'



Loading Linux 5.10.0-19-cloud-amd64 ...Loading Linux 5.10.0-19-cloud-amd64 ...

Loading initial ramdisk ...Loading initial ramdisk ...

error: no suitable video mode found.
error: no suitable video mode found.
Booting in blind modeBooting in blind mode

[    0.000000] Linux version 5.10.0-19-cloud-amd64 (debian-kernel@lists.debian.org) (gcc-10 (Debian 10.2.1-6) 10.2.1 20210110, GNU ld (GNU Binutils for Debian) 2.35.2) #1 SMP Debian 5.10.149-2 (2022-10-21)
[    0.000000] Command line: BOOT_IMAGE=/boot/vmlinuz-5.10.0-19-cloud-amd64 root=UUID=c9

In [21]:
LOCAL_PATH_TO = "/home/eithanm"
print (f"gcloud compute scp {LOCAL_PATH_TO}/search_frontend.py {GOOGLE_ACCOUNT_NAME}@{INSTANCE_NAME}:/home/{GOOGLE_ACCOUNT_NAME}")

gcloud compute scp /home/eithanm/search_frontend.py eithanm@instance-1:/home/eithanm


In [16]:
# 4. Secure copy your app to the VM
!gcloud compute scp LOCAL_PATH_TO/search_frontend.py $GOOGLE_ACCOUNT_NAME@$INSTANCE_NAME:/home/$GOOGLE_ACCOUNT_NAME

This tool needs to create the directory [/root/.ssh] before being able to 
generate SSH keys.

Do you want to continue (Y/n)?  ^C


Command killed by keyboard interrupt



In [22]:
print(f'gcloud compute ssh {GOOGLE_ACCOUNT_NAME}@{INSTANCE_NAME}')
# gcloud compute ssh $GOOGLE_ACCOUNT_NAME@$INSTANCE_NAME
# 5. SSH to your VM and start the app


gcloud compute ssh eithanm@instance-1


In [ ]:
# python3 search_frontend.py

In [ ]:


################################################################################
# Clean up commands to undo the above set up and avoid unnecessary charges
gcloud compute instances delete -q $INSTANCE_NAME
# make sure there are no lingering instances
gcloud compute instances list
# delete firewall rule
gcloud compute firewall-rules delete -q default-allow-http-8080
# delete external addresses
gcloud compute addresses delete -q $IP_NAME --region $REGION